In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

data = pd.read_csv("data.tsv", skipinitialspace = True, sep = "\t")

#drop non-relevant rows
data.drop(data.tail(5).index, inplace=True)

#let's look at the data
data.head()

,Kooli ID,Kooli nimi,Maakond,Asutuse tüüp,Seadmete nimekiri,Toetus 2014,Toetus 2015,Toetus 2016,Toetus 2017,Toetus 2018,Toetus 2019,Toetus 2020,Toetus 2021
0,437,Abja Gümnaasium,Viljandi maakond,gümnaasium,"LEGO Mindstorm EV3 baaskomplekt (9 tk), lisako...","3,000.00 €",NaN,NaN,NaN,NaN,"2,154.24 €",NaN,NaN
1,335,Antsla Gümnaasium,Võru maakond,gümnaasium,"LEGO Mindstorms EV3 baaskomplekt (4 tk), EV3 l...",NaN,"1,279.20 €",NaN,NaN,"1,589.00 €",NaN,NaN,NaN
2,199,Aravete Keskkool,Järva maakond,gümnaasium,"LEGO Mindstorms EV3 baaskomplekt (8 tk), EV3 l...",NaN,"3,535.26 €",NaN,NaN,"1,880.35 €",NaN,"2,857.68 €","4,410 €"
3,256,August Kitzbergi nimeline Gümnaasium,Viljandi maakond,gümnaasium,Kodulabor III (1 tk) ja LEGO Mindstorms EV3 b...,"1,151.40 €",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,597,Avinurme Gümnaasium,Jõgeva maakond,gümnaasium,"LEGO Mindstorms EV3 baaskomplekt (3 tk), EV3 l...",NaN,"1,146.53 €",NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
#Translations of column names:

#Kooli ID - school's ID
#Maakond - county
#Asutuse tüüp - Type of school (high school, middle school, vocational school etc)
#Seadmete nimekiri - list of devices
#Toetus - funding

In [3]:
#basic check for possible typos in some columns
print("Maakonnad:")
#these values were also used to 
data['Maakond'].value_counts()
print("Asutused:")
data['Asutuse tüüp'].value_counts()

Maakonnad:


Harju maakond         201
Tartu maakond          96
Ida-Viru maakond       64
Pärnu maakond          60
Lääne-Viru maakond     41
Viljandi maakond       33
Rapla maakond          32
Võru maakond           28
Jõgeva maakond         25
Järva maakond          24
Põlva maakond          23
Lääne maakond          22
Saare maakond          18
Valga maakond          15
Hiiu maakond            9
Name: Maakond, dtype: int64

Asutused:


lasteaed                    301
põhikool                    196
gümnaasium                  124
lasteaed-põhikool            59
kutseõppeasutus              10
täiskasvanute gümnaasium      1
huvikool                      1
Name: Asutuse tüüp, dtype: int64

In [13]:
#remove unneccessary symbols from the funding amounts
def formatnumber(number):
    number=str(number).replace(',', '').replace('€', '').strip()
    return float(number)

#make a dictionary, where the keys are names of counties and values dictionaries, where keys are years and values amounts
#county:(year:amount)

def getyearlyfundings(data):
    dictionary = {}
    
    #for rows
    for i in range(len(data)):
        county = str(data.iloc[i, 2]).split(" ")[0]
        if county in dictionary:
            years = dictionary[county]
        else:
            years = {}
        
        #for each year
        for j in range(8):
            year = 2014+j
            cell = data.iloc[i, j+5]
            if pd.isna(cell):
                continue
            amount = formatnumber(cell)
            if year not in years: 
                years[year] = amount
            else:
                years[year] = years[year] + amount
        dictionary[county] = years
    return dictionary


fundings = getyearlyfundings(data)
#add to dataframe and flip axes
df = pd.DataFrame(data=fundings).T
#reorder columns by years and rows alphabetically
df.sort_index(axis=1, inplace=True)
df.sort_index(inplace=True)
df

#df.to_csv('mapinfo.csv')
#Thiscsv file was used to create the maps. Some additional calculations were made inside the file.

#last row is nan because row 176 in 'data' dataframe is 'Klooga lasteaed', which doesn't have an index nor county, but says
#it's closed. The authors assumed devices were returned and didn't account them. Even if it wasn't the case, the funding was
#only 728.32€, which is not significant compared to the total sums.

,2014,2015,2016,2017,2018,2019,2020,2021
Harju,45922.23,72904.93,78437.17,74521.36,65249.29,57712.65,105765.97,89713.95
Hiiu,805.48,612.00,3234.56,2842.78,526.68,1647.09,6070.42,NaN
Ida-Viru,8912.90,22497.27,16607.28,14632.83,16568.62,31154.87,41876.75,13348.34
Järva,5019.51,16602.05,10041.90,7603.04,4514.43,22313.44,12321.15,11130.30
Jõgeva,8725.47,11958.86,7121.92,8397.15,3371.13,9661.59,10971.81,13602.70
Lääne,1424.28,10340.97,8461.36,6464.64,11995.69,5274.71,5873.37,5933.18
Lääne-Viru,15817.65,15159.50,4716.53,14942.72,8330.38,18821.96,12399.58,25593.07
Pärnu,9877.63,13261.90,13548.11,24863.84,20327.42,24028.27,29234.91,19176.31
Põlva,1402.33,3193.71,2800.43,8527.01,9492.91,6331.63,7896.20,7240.98
Rapla,2519.73,12819.55,12758.01,3691.64,16188.61,12206.88,5591.67,10948.38


In [9]:
#Last row in our source data file has the total sums per year. It was discovered that they are not correct as they were
#calculated again here and in the Excel workbook as well, to double check. The author of the Excel has left the first
#four rows out of the calculation for some reason.

for i in range(8):
    print("Year is " + str(2014 + i))
    yearlysum = 0
    for j in range(len(data)):
          cell = data.iloc[j, i+5]
          if pd.isna(cell):
                continue
          yearlysum += formatnumber(cell)
    print("Sum is " + str(yearlysum))

Year is 2014
Sum is 164009.30000000005
Year is 2015
Sum is 231278.97000000006
Year is 2016
Sum is 190422.31
Year is 2017
Sum is 244105.81
Year is 2018
Sum is 239748.94000000003
Year is 2019
Sum is 289976.3800000001
Year is 2020
Sum is 292010.5100000001
Year is 2021
Sum is 295156.48000000004
